In [2]:
!pip install llama-index llama-index-readers-web duckduckgo_search

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.5/15.5 MB 27.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 38.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.5/328.5 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 27.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 kB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.1/211.1 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.9/37.9 MB 27.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 40.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 55.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

# Identify specific region(s) that a location belongs to (used to match location to corresponding data regulation policies)

### Define locations and link to regulations

In [3]:
def get_privacy_law_url(region_name):
    privacy_laws = {
        "United States": "https://www.ftc.gov/legal-library/browse/statutes/ftc-act",
        "California": "https://cppa.ca.gov/regulations/pdf/cppa_act.pdf",
        "European Union": "https://eur-lex.europa.eu/legal-content/EN/TXT/PDF/?uri=CELEX:32016R0679",
        "Canada": "https://laws-lois.justice.gc.ca/pdf/p-8.6.pdf",
        "Australia": "https://www.oaic.gov.au/__data/assets/pdf_file/0009/1125/app-guidelines-july-2019.pdf",
        "United Kingdom": "https://www.gov.uk/data-protection",
        "Japan": "https://www.cas.go.jp/jp/seisaku/hourei/data/APPI.pdf",
        "South Korea": "https://www.pipc.go.kr/cmt/main/english/main.html",
        "Brazil": "http://www.planalto.gov.br/ccivil_03/_ato2015-2018/2018/lei/L13709.htm",
        "India": "https://www.meity.gov.in/content/privacy-data-protection",
        "South Korea": "http://koreanlii.or.kr/w/images/0/0e/KoreanDPAct2011.pdf",
        "South Africa": "https://www.gov.za/sites/default/files/gcis_document/201409/3706726-11act4of2013protectionofpersonalinforcorrect.pdf",
        "Singapore": "https://www.pdpc.gov.sg/-/media/files/pdpc/pdf-files/advisory-guidelines/ag-on-key-concepts/advisory-guidelines-on-key-concepts-in-the-pdpa-17-may-2022.pdf",
        "Brazil":"https://www.lgpdbrasil.com.br/wp-content/uploads/2019/06/LGPD-english-version.pdf",
    }

    return privacy_laws.get(region_name, "Privacy law URL not found for this region")

### LLM (identify location)

In [4]:
import os

import regex as re
import pandas as pd
from llama_index.llms.openai import OpenAI
from llama_index.core import Settings
from llama_index.core import SummaryIndex, VectorStoreIndex
from llama_index.readers.web import SimpleWebPageReader
from llama_index.core.tools import BaseTool, FunctionTool
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.core.query_engine import SubQuestionQueryEngine
from llama_index.core.agent import ReActAgent
from IPython.display import Markdown, display
from duckduckgo_search import DDGS
os.environ["OPENAI_API_KEY"] = "sk-proj-e20Q6bKuRvBVQBObIFhvT3BlbkFJ3HdudAgJER9AnPxwsgFp"

In [5]:
import nest_asyncio

nest_asyncio.apply()

In [6]:
token = 'sk-proj-e20Q6bKuRvBVQBObIFhvT3BlbkFJ3HdudAgJER9AnPxwsgFp'
llm = OpenAI(model="gpt-4o", openai_api_key=token)
Settings.llm = llm

In [7]:
REGION_TEMPLATE = """
    Assistant: I can help determine which region a user input belongs to. The options are: United States, California, European Union, Canada, Australia, United Kingdom, Japan, South Korea, South Africa, Singapore, Brazil, Other. Make sure to include all options.
    User: {user_input}
    Assistant: The regions that the user input belongs to is (are):
    Example:
    User: los angeles
    Assistant: California, United States.
    User: hong kong
    Assistant: Other.
    """

In [8]:
def determine_region(user_input, api_key, temperature=0.3):
    # Initialize the OpenAI model with llama-index
    llm = OpenAI(model="gpt-4o", openai_api_key=api_key)

    REGION_TEMPLATE = """
    Assistant: I can help determine which region a user input belongs to. The options are: United States, California, European Union, Canada, Australia, United Kingdom, Japan, South Korea, South Africa, Singapore, Brazil, Other. Make sure to include all options.

    Examples:
    User: los angeles
    Assistant: California, United States
    User: hong kong
    Assistant: Other
    User: berlin
    Assistant: European Union
    User: toronto
    Assistant: Canada
    Now, let's try with your input.
    User: {user_input}
    Assistant: The regions that the user input belongs to are:
    """

    prompt = REGION_TEMPLATE.format(user_input=user_input)

    response = llm.complete(prompt=prompt, max_tokens=50, temperature=temperature)

    answer = response.text.strip().replace("The regions that the user input belongs to is (are):", "").strip()

    return answer

In [18]:
def clean_region_output(text):
  return text.split('\n')[0]

In [24]:
def get_privacy_law_links(user_input, api_key = "sk-proj-e20Q6bKuRvBVQBObIFhvT3BlbkFJ3HdudAgJER9AnPxwsgFp"):
    regions = determine_region(user_input, api_key)
    regions = clean_region_output(regions)
    region_list = regions.split(', ') # Assuming regions are comma-separated
    links = []
    for region in region_list:
        link = get_privacy_law_url(region.strip())
        links.append(link)
    return region_list, links

In [25]:
la_regions, la_links = get_privacy_law_links("los angeles")
la_links

['https://cppa.ca.gov/regulations/pdf/cppa_act.pdf',
 'https://www.ftc.gov/legal-library/browse/statutes/ftc-act']

In [12]:
tokyo_links = get_privacy_law_links("tokyo")
tokyo_links

['https://www.cas.go.jp/jp/seisaku/hourei/data/APPI.pdf']

In [14]:
beijing_links = get_privacy_law_links("beijing")
beijing_links

['Privacy law URL not found for this region']

In [26]:
for i in range(10):
  print(get_privacy_law_links("los angeles"))

(['California', 'United States'], ['https://cppa.ca.gov/regulations/pdf/cppa_act.pdf', 'https://www.ftc.gov/legal-library/browse/statutes/ftc-act'])
(['California', 'United States'], ['https://cppa.ca.gov/regulations/pdf/cppa_act.pdf', 'https://www.ftc.gov/legal-library/browse/statutes/ftc-act'])
(['California', 'United States'], ['https://cppa.ca.gov/regulations/pdf/cppa_act.pdf', 'https://www.ftc.gov/legal-library/browse/statutes/ftc-act'])
(['California', 'United States'], ['https://cppa.ca.gov/regulations/pdf/cppa_act.pdf', 'https://www.ftc.gov/legal-library/browse/statutes/ftc-act'])
(['California', 'United States'], ['https://cppa.ca.gov/regulations/pdf/cppa_act.pdf', 'https://www.ftc.gov/legal-library/browse/statutes/ftc-act'])
(['California', 'United States'], ['https://cppa.ca.gov/regulations/pdf/cppa_act.pdf', 'https://www.ftc.gov/legal-library/browse/statutes/ftc-act'])
(['California', 'United States'], ['https://cppa.ca.gov/regulations/pdf/cppa_act.pdf', 'https://www.ftc.go

In [21]:
for i in range(10):
  print(get_privacy_law_links("san diego"))

California, United States
['https://cppa.ca.gov/regulations/pdf/cppa_act.pdf', 'https://www.ftc.gov/legal-library/browse/statutes/ftc-act']
California, United States
['https://cppa.ca.gov/regulations/pdf/cppa_act.pdf', 'https://www.ftc.gov/legal-library/browse/statutes/ftc-act']
California, United States
['https://cppa.ca.gov/regulations/pdf/cppa_act.pdf', 'https://www.ftc.gov/legal-library/browse/statutes/ftc-act']
California, United States
['https://cppa.ca.gov/regulations/pdf/cppa_act.pdf', 'https://www.ftc.gov/legal-library/browse/statutes/ftc-act']
California, United States
['https://cppa.ca.gov/regulations/pdf/cppa_act.pdf', 'https://www.ftc.gov/legal-library/browse/statutes/ftc-act']
California, United States
['https://cppa.ca.gov/regulations/pdf/cppa_act.pdf', 'https://www.ftc.gov/legal-library/browse/statutes/ftc-act']
California, United States
['https://cppa.ca.gov/regulations/pdf/cppa_act.pdf', 'https://www.ftc.gov/legal-library/browse/statutes/ftc-act']
California, United S

In [23]:
la_reader = SimpleWebPageReader(html_to_text=True).load_data(la_links)

In [28]:
la_data_regulation_query_tool = [
    QueryEngineTool(
        query_engine=VectorStoreIndex.from_documents([la_reader[i]]).as_query_engine(),
        metadata=ToolMetadata(
            name=f"data_regulation_for_{la_regions[i]}",
            description=("useful for when you want to know "
                         f"{la_regions[i]}'s data regulation"),
        ),
    ) for i in range(len(la_links))
]

In [30]:
la_data_regulation_query_tool[0].query_engine.query('What is the data protection law for California about?').response

'The data protection law for California, known as the California Consumer Privacy Act of 2018, focuses on providing privacy rights and consumer protection for residents of California. It grants consumers the right to know what personal data is being collected about them, to whom it is being sold or disclosed, and the ability to access, delete, and opt-out of the sale of their personal information.'

In [32]:
la_data_regulation_query_engine = SubQuestionQueryEngine.from_defaults(
    query_engine_tools=la_data_regulation_query_tool)

In [36]:
la_data_regulation_query_engine_tool = QueryEngineTool(
    query_engine=la_data_regulation_query_engine,
    metadata=ToolMetadata(
        name="sub_question_query_engine_for_data_regulation",
        description=(
            "useful for when you want to answer queries that require analyzing"
            f" multiple data regulations from different regions, including: {', '.join(la_regions)}"),
    ),
)

In [38]:
la_data_regulation_query_engine_tool.query_engine.query('What are the common elements of the data protection laws for los angeles?').response

Generated 2 sub questions.
[data_regulation_for_California] Q: What are the data protection laws specific to California?
[data_regulation_for_United States] Q: What are the data protection laws specific to the United States?
[data_regulation_for_California] A: The data protection law specific to California is the California Consumer Privacy Act of 2018.
[data_regulation_for_United States] A: To find information on data protection laws specific to the United States, you can search or browse the Legal Library on the relevant government website. This resource provides legal guidance and documents to help understand business responsibilities and compliance with the law.


'Common elements of data protection laws for Los Angeles would include compliance with the California Consumer Privacy Act of 2018, which is a state-wide regulation. This act focuses on consumer rights regarding personal data, including the right to know what data is being collected, the right to delete personal data, and the right to opt-out of the sale of personal data. Additionally, businesses must provide transparency about their data collection practices and ensure the protection of consumer data.'

In [51]:
def get_data_regulation_info(location, api_key="sk-proj-e20Q6bKuRvBVQBObIFhvT3BlbkFJ3HdudAgJER9AnPxwsgFp"):
    """
    This function takes a location as input and returns a query engine tool
    that can be used to answer questions about data regulations in that location.
    """
    regions, links = get_privacy_law_links(location, api_key)
    if 'Privacy law URL not found for this region' in links:
        print('Data regulation law URL not found for this location.')
        return None
    reader = SimpleWebPageReader(html_to_text=True).load_data(links)
    data_regulation_query_tool = [
        QueryEngineTool(
            query_engine=VectorStoreIndex.from_documents([reader[i]]).as_query_engine(),
            metadata=ToolMetadata(
                name=f"data_regulation_for_{regions[i]}",
                description=("useful for when you want to know "
                             f"{regions[i]}'s data regulation"),
            ),
        ) for i in range(len(links))
    ]
    data_regulation_query_engine = SubQuestionQueryEngine.from_defaults(
        query_engine_tools=data_regulation_query_tool)
    data_regulation_query_engine_tool = QueryEngineTool(
        query_engine=data_regulation_query_engine,
        metadata=ToolMetadata(
            name="sub_question_query_engine_for_data_regulation",
            description=(
                "useful for when you want to answer queries that require analyzing"
                f" multiple data regulations from different regions, including: {', '.join(regions)}"),
        ),
    )
    return data_regulation_query_engine_tool


In [46]:
location = "los angeles"
query_engine_tool = get_data_regulation_info(location)
response = query_engine_tool.query_engine.query('What are the common elements of the data protection laws for los angeles?').response
print(response)

Generated 2 sub questions.
[data_regulation_for_California] Q: What are the key elements of California's data regulation?
[data_regulation_for_United States] Q: What are the key elements of United States's data regulation?
[data_regulation_for_United States] A: The key elements of the United States' data regulation include the enforcement of federal competition and consumer protection laws to prevent anticompetitive, deceptive, and unfair business practices. Additionally, there are specific programs and sections such as the Premerger Notification Program, Merger Review, and Anticompetitive Practices. The regulation also involves issuing warning letters, managing the Consumer Sentinel Network, and overseeing the FTC Refund Programs. Guidance documents and policy statements are provided to help businesses understand and comply with these laws.
[data_regulation_for_California] A: The key elements of California's data regulation include the protection of consumer privacy, the establishment

In [47]:
location = "tokyo"
query_engine_tool = get_data_regulation_info(location)
response = query_engine_tool.query_engine.query(f'What are the common elements of the data protection laws for {location}?').response
print(response)

Generated 5 sub questions.
[data_regulation_for_Japan] Q: What are the key principles of data protection laws in Tokyo?
[data_regulation_for_Japan] Q: What are the requirements for data processing under Tokyo's data protection laws?
[data_regulation_for_Japan] Q: What are the rights of individuals under Tokyo's data protection laws?
[data_regulation_for_Japan] Q: What are the penalties for non-compliance with data protection laws in Tokyo?
[data_regulation_for_Japan] Q: Are there any specific regulations for cross-border data transfers in Tokyo's data protection laws?
[data_regulation_for_Japan] A: I'm sorry, but I can't provide the information you're looking for.
[data_regulation_for_Japan] A: I cannot provide specific details about Tokyo's data protection laws based on the given context.
[data_regulation_for_Japan] A: The penalties for non-compliance with data protection laws in Tokyo can include fines, administrative sanctions, and potential legal action. Specific penalties may vary

In [52]:
location = "Shenzhen"
query_engine_tool = get_data_regulation_info(location)
response = query_engine_tool.query_engine.query(f'What are the common elements of the data protection laws for {location}?').response
print(response)

Data regulation law URL not found for this location.


AttributeError: 'NoneType' object has no attribute 'query_engine'